In [5]:
import re
import subprocess
from datetime import datetime
from pathlib import Path

# === 檔案路徑 ===
input_path = Path("papers.bib")
output_path = Path("papers_dedup.bib")
docx_output_path = Path("【借名登記】文獻清單匯出.docx")

def clean_entry(entry: str) -> str:
    """刪除值為『未知』『無』或空 {} 的欄位"""
    return re.sub(
        r'^\s*\w+\s*=\s*\{\s*(?:未知|無)?\s*\},?\s*$',
        '',
        entry,
        flags=re.MULTILINE
    )

def parse_entry(entry):
    """解析 Bib 條目"""
    m_type = re.match(r'(\w+)\s*\{', entry)
    m_title = re.search(r'title\s*=\s*\{(.*?)\}', entry, re.DOTALL)
    m_author = re.search(r'author\s*=\s*\{(.*?)\}', entry, re.DOTALL)
    m_year = re.search(r'year\s*=\s*\{(.*?)\}', entry)
    typ = m_type.group(1) if m_type else "unknown"
    title = m_title.group(1).strip() if m_title else "未知標題"
    author = m_author.group(1).strip() if m_author else "未知作者"
    year = m_year.group(1).strip() if m_year else "未知年份"
    cleaned_entry = clean_entry(entry)
    return typ, title, author, year, cleaned_entry

def extract_field(entry: str, field_name: str):
    """提取指定欄位"""
    pattern = rf'{field_name}\s*=\s*\{{(.*?)\}}'
    match = re.search(pattern, entry, flags=re.DOTALL | re.IGNORECASE)
    if match:
        value = match.group(1).strip()
        return value or None
    return None

def collect_howpublish(entries):
    """收集 howpublish 欄位"""
    values = []
    for _, _, _, entry in entries:
        howpublish = extract_field(entry, "howpublished") or extract_field(entry, "howpublish")
        if howpublish:
            values.append(howpublish)
    return values

def write_howpublish_docx(values, output_path: Path):
    """用 Pandoc 輸出 DOCX"""
    md_path = output_path.with_suffix(".md")
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    lines = [
        "# 文獻清單 匯出",
        f"- 產出時間：{timestamp}",
        "",
        "## 條目清單",
        ""
    ] + [f"- {v}" for v in values]

    md_path.write_text("\n".join(lines), encoding="utf-8")

    subprocess.run([
        "pandoc",
        str(md_path),
        "-f", "markdown",
        "-t", "docx",
        "-o", str(output_path)
    ], check=True)

    print(f"\n✅ 已輸出 howpublish DOCX（共 {len(values)} 筆）：{output_path}")

# === 主流程 ===
text = input_path.read_text(encoding="utf-8")
entries = re.split(r'@(?=\w+\s*\{)', text)
entries = [e.strip() for e in entries if e.strip()]

parsed = [parse_entry(e) for e in entries]

# === 去重 ===
seen = {}
duplicates = []

for typ, title, author, year, entry in parsed:
    key = title.lower()
    if key in seen:
        duplicates.append((title, author, year))
    else:
        seen[key] = (title, author, year, entry)

unique_entries = list(seen.values())

# === 統計輸出 ===
total_count = len(parsed)
dup_count = len(duplicates)
unique_count = len(unique_entries)

print(f"原始篇數：{total_count}")
print(f"重複篇數：{dup_count}")
if dup_count:
    print("\n重複條目：")
    for title, author, year in duplicates:
        print(f" - {title} / {author} / {year}")
print(f"\n去重後篇數：{unique_count}")
print("\n去重後條目：")
for title, author, year, _ in unique_entries:
    print(f" - {title} / {author} / {year}")

# === 寫出去重後 Bib ===
dedup_text = "\n\n".join(f"@{entry}" for _, _, _, entry in unique_entries)
dedup_text = re.sub(r'\n{3,}', '\n\n', dedup_text)
output_path.write_text(dedup_text, encoding="utf-8")
print(f"\n✅ 已輸出去重後檔案：{output_path}")

# === 匯出 howpublish DOCX ===
howpublish_entries = collect_howpublish(unique_entries)
if howpublish_entries:
    write_howpublish_docx(howpublish_entries, docx_output_path)
else:
    print("\n⚠️ 去重後條目未找到 howpublish 欄位，未產生 DOCX。")

原始篇數：35
重複篇數：4

重複條目：
 - 借名登記之相關法律效力－最高法院 108 年度台上大字第 1652 號裁定 / 王怡蘋 / 2022
 - 原住民保留地「借名登記契約」的效力－公法規定的私法效力，或私法自治的公法限制？ / 李建良 / 2021
 - 不動產借名登記契約有效性的檢討 / 黃健彰 / 2019
 - 不動產借名登記契約之負擔行為與處分行為 / 蔡旻耿 and 陳靜瑩 / 2016

去重後篇數：31

去重後條目：
 - 不動產借名登記相關民事法律關係－以評析近年最高法院裁判為主 / 黃健彰 / 2025
 - 民事法裁判精選－借名登記之內部關係與外部關係（111 台上 2686 判決） / 顏佑紘 / 2024
 - 論借名登記 / 蔣瑞安 / 2023
 - 借名登記股份的回復與公同共有－最高法院 110 年度台上字第 724 號民事判決 / 陳榮傳 / 2022
 - 原住民保留地的借名登記－大法庭裁定的商榷 / 陳榮傳 / 2022
 - 自耕能力與借名登記－最高法院 107 年度台上字第 1023 號判決評析 / 林慶郎 / 2022
 - 借名登記之舉證責任 / 鄭冠宇 / 2022
 - 借名登記之相關法律效力－最高法院 108 年度台上大字第 1652 號裁定 / 王怡蘋 / 2022
 - 原住民保留地借名登記之法律關係－兼評最高法院 108 年台上大字第 1636 號民事裁定 / 蔡旻耿 / 2022
 - 原住民保留地「借名登記契約」的效力－公法規定的私法效力，或私法自治的公法限制？ / 李建良 / 2021
 - 農地借名登記問題探討－以刪除前土地法第 30 條適用範圍及當事人間契約有效性為核心 / 李維中 / 2021
 - 淺談商標借名登記 / 涂軼 / 2020
 - 不動產借名登記契約有效性的檢討 / 黃健彰 / 2019
 - 我國不動產借名登記後處分之效力問題分析－以最高法院 106 年第 3 次民事庭會議決議為中心 / 陳明燦 / 2018
 - 借名登記契約是否屬「因委任事務之性質不能消滅者」之實務爭議－最高法院 106 年度台上字第 410 號民事判決評析 / 邱玟惠 / 2018
 - 借名登記對外效力之探討－法學方法與債法現代化之思考 / 林大洋 / 2017
 - 借名登記財產之請求返還